In [8]:
# !python -m spacy download en_core_web_sm

In [4]:
#Build tokenizer
def tokenizer(text):
    return [token.text for token in spacy_en.tokenizer(text)]

In [5]:
from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

In [7]:
print(tokenizer("I can't run whole day"))

['I', 'ca', "n't", 'run', 'whole', 'day']


In [14]:
# from torchtext.data import *
# TEXT = data.Field(tokenize=tokenizer, use_vocab=True, lower=True, batch_first=True, include_lengths=True)
# LABEL = data.LabelField(dtype=torch.long, batch_first=True, sequential=False)
# fields = [('text', TEXT), ('label', LABEL)]

In [15]:
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.nn import functional as F

class CustomTextDataset(Dataset):
    def __init__(self, text, labels):
        self.labels = labels
        self.text = text
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        sample = {"Text": text, "labels": label}
        return sample

TrainingDataObj = CustomTextDataset(train_data['text'], train_data['label'])
TestDataObj= CustomTextDataset(test_data['text'], test_data['label'])

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    
    for asample in batch:
        label_list.append(label_pipeline(asample['labels']))
        #print (label_list)
        processed_text = torch.tensor(text_pipeline(asample['Text']) , dtype=torch.int64)  
        #print (processed_text.size())
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    #text_list = torch.stack(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

Traindataloader = DataLoader(TrainingDataObj, batch_size=5, shuffle=True, collate_fn= collate_batch)

for idx, (label, text, offsets) in enumerate(Traindataloader):
    print(text.size())
    print(label)
    print(offsets.int())
    break
    


NameError: name 'torch' is not defined

In [2]:
import pandas as pd

In [3]:
# train data
df_prepro_train = pd.read_csv('processed_train.csv',na_filter=False)
# Train data
X_trn = df_prepro_train['Pre Processed Text']
y_trn = df_prepro_train['Class Labels']

In [4]:
# from torchtext.data import datasets_utils

In [5]:
# y_trn

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y_trn)

In [1]:
import torchtext
import os
import torch
import pandas as pd
# from skimage import io, transform
import numpy as np
# import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms, utils

In [2]:
# df = pd.DataFrame({"Text":X_train,'Label':y})

In [3]:
# df

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_trn, y, test_size=0.33, random_state=42)

C:\Users\KHADGA JYOTH ALLI\anaconda3\envs\gpu\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


NameError: name 'X_trn' is not defined

In [ ]:
df = pd.DataFrame({"Text":X_train,'Label':y_train})
df.reset_index(drop=True,inplace=True)
df_test = pd.DataFrame({"Text":X_test,'Label':y_test})
df_test.reset_index(drop=True,inplace=True)


In [ ]:
# df

In [6]:
#Now we need to create a vocabulary from our train text
#We need two things (i) A tokenizer that will tokenize a given text (ii) A counter object that will count occurance of tokens

from torchtext.data import get_tokenizer
import collections

tokenizer = get_tokenizer("basic_english")
counter_obj = collections.Counter()

for atext in df['Text']:
    tokens = tokenizer(atext)
    counter_obj.update(tokens)
    
from torchtext.vocab import vocab

vocabobj = vocab(counter_obj, min_freq = 1, specials=['<unk>'] )
vocabobj.set_default_index(vocabobj['<unk>'])
vocabobj

NameError: name 'df' is not defined

In [7]:
def trp(l, n):
    return l[:n] + [0]*(n-len(l))

inp_len = 100
text_pipeline = lambda x: trp(vocabobj(tokenizer(x)),inp_len)
label_pipeline = lambda x: x

for aline in df['Text']:
    print (len(text_pipeline(aline)))
    break

NameError: name 'df' is not defined

In [15]:

from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.nn import functional as F
class NLPDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        self.text = data['Text']
        self.labels = data['Label']
        # self.len_data = data.shape[0]

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # img_name = os.path.join(self.root_dir,
        #                         self.landmarks_frame.iloc[idx, 0])
        # image = io.imread(img_name)
        # landmarks = self.landmarks_frame.iloc[idx, 1:]
        # landmarks = np.array([landmarks])
        # landmarks = landmarks.astype('float').reshape(-1, 2)
        # data = self.data.iloc[idx]
        text = self.text[idx]
        label = self.labels[idx]
        sample = {'labels': label, 'Text': text}

        return sample

In [16]:
TrainingDataObj = NLPDataset(df)
TestDataObj= NLPDataset(df_test)
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    
    for asample in batch:
        label_list.append(label_pipeline(asample['labels']))
        #print (label_list)
        processed_text = torch.tensor(text_pipeline(asample['Text']) , dtype=torch.int64)  
        #print (processed_text.size())
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    #text_list = torch.stack(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)
Traindataloader = DataLoader(TrainingDataObj, batch_size=5, shuffle=True, collate_fn= collate_batch)

for idx, (label, text, offsets) in enumerate(Traindataloader):
    print(text.size())
    print(label)
    print(offsets.int())
    break
    

torch.Size([500])
tensor([0, 1, 0, 1, 0])
tensor([  0, 100, 200, 300, 400], dtype=torch.int32)


In [17]:
TrainingDataObj[0]

{'labels': 0,
 'Text': 'away day view parties everyone watch match sunday get bus come watch'}

In [18]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [5]:
torch.cuda.is_available()

True

In [19]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cpu')

In [18]:
num_class = len(set(TrainingDataObj.labels))
vocab_size = len(vocabobj)
emsize = 64
input_size = inp_len
hidden_size=20
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [19]:
len(vocabobj)

142524

In [20]:
import time
from tqdm import tqdm
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 1000
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count


In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None


train_dataloader = DataLoader(TrainingDataObj, batch_size=256, shuffle=True, collate_fn= collate_batch)
test_dataloader = DataLoader(TestDataObj, batch_size=256, shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(test_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'test accuracy {:8.3f} '.format(epoch, time.time() - epoch_start_time, accu_val))
    print('-' * 59)
